In [ ]:
import os
import uuid

from IPython.display import display

import folium
from folium.plugins import MarkerCluster

import openrouteservice

import time
import pandas as pd
import geopandas as gpd
import fiona as fn
from shapely.geometry import shape, mapping
from shapely.ops import cascaded_union

from tqdm.notebook import tqdm

In [ ]:
#load centroids or stops, depending on the isochrones you want to create 
stations = "bus_stops.gpkg"
df = gpd.read_file(stations)

In [ ]:
df  = df.to_crs({'init': 'epsg:4326'})

## Create Isochrones

In [ ]:
from tqdm.notebook import tqdm
from openrouteservice import client 
import geopandas as gpd

normal_isochrones = []
err_iso = []


#You also need to modify the isochrones parameters. 
#If you are calculating normal isochrones (no rush), you need to set 'time' as range_type, and write 3600 (30 minutes) in range.
#If you are calculating rush hour isochrones, you need to set 'distance' as range_type and enter the distance in range in meters. For Lima, we calculated that during rush hour, you can drive 9380 meters while for London, it's 13640. 

profile = 'driving-car'
range_type = 'distance'


isochrones_parameters = {
    'profile': profile,
    'range_type': range_type,
    'range': [13640],
}




ors_client = client.Client(base_url='http://localhost:8081/ors')

# Use tqdm to wrap the iteration over the GeoDataFrame
for _, row in tqdm(df.iterrows(), total=len(df)):

    try:
        coordinate = (row.geometry.x, row.geometry.y)
        stop_id = row['uuid']  # Extract stop_id for the current row
    
        # Calculate isochrone for the single coordinate
        isochrones_parameters['locations'] = [coordinate]  # Ensure it's a list with one coordinate
        isochrone_normal = ors_client.isochrones(**isochrones_parameters)
        isochrone_normal_df = gpd.GeoDataFrame().from_features(isochrone_normal)
        # Add stop_id column to the isochrone GeoDataFrame
        isochrone_normal_df['uuid'] = stop_id
        # Append the single isochrone to the list
        normal_isochrones.append(isochrone_normal_df)
        
    except:
        print(row['uuid'])
        err_iso.append(row)
        continue

## Save the dataset as a geopackage

In [ ]:
#Save with a chosen name
isochrones_all = pd.concat(normal_isochrones).set_crs(epsg=4326)
isochrones_all

isochrones_all.to_file("isochrones_normal.gpkg") #or isochrones_rush